In [2]:
import numpy as np
import pandas as pd
from keras.datasets import mnist
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier
from numpy import mean
from numpy import std
from sklearn.model_selection import KFold
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
# application (model) imports
from keras import applications
import tensorflow as tf

Using TensorFlow backend.


In [3]:
(trainX, trainY), (testX, testY) = tf.keras.datasets.cifar100.load_data(label_mode="fine")

169009152/169001437 [==============================] - 114s 1us/step


In [4]:
print(trainX.shape)

(50000, 32, 32, 3)


In [5]:
testY = to_categorical(testY)
trainY = to_categorical(trainY)

In [6]:
#Reshaping the data
trainX = trainX.astype('float32')
trainX = trainX / 255.0

#Reshaping the data
testX = testX.astype('float32')
testX = testX / 255.0

In [7]:
def plot_loss_accuracy(history):
    historydf = pd.DataFrame(history.history, index=history.epoch)
    plt.figure(figsize=(8, 6))
    historydf.plot(ylim=(0, max(1, historydf.values.max())))
    loss = history.history['loss'][-1]
    acc = history.history['accuracy'][-1]
    plt.title('Loss: %.3f, Accuracy: %.3f' % (loss, acc))

def dir_to_array(dir, image_shape, verbose=1):
  data = []
  labels = []
  image_shape = (image_shape[0], image_shape[1])
  if verbose ==1:
    bar = progressbar.ProgressBar(maxval=len(os.listdir(dir)),
                                  widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    i = 0
  for classname in os.listdir(dir):
    class_dir = os.path.join(dir, classname)
    for imagename in os.listdir(class_dir):
      data.append(np.array(load_img(os.path.join(class_dir, imagename), target_size=image_shape )))
      labels.append(int(classname)-1)
    if verbose == 1:
      i +=1
      bar.update(i)
  if verbose == 1:
    bar.finish()
  data = np.stack(data, axis=0)
  labels = np.expand_dims(np.stack(labels, axis=0), axis=1)
  return data, labels


def network_library(network_name, num_classes, weights='imagenet'):
  # Function to build keras applications 
  if network_name == 'vgg16':
    
    input_shape = (256,256,3)
    base_model = applications.vgg16.VGG16(include_top=False, weights=weights, input_shape=input_shape, pooling='max')
    preprocess_input = applications.vgg16.preprocess_input
    
    x = base_model.output
    x = Dense(4096, activation='relu')(x)
    x = Dense(4096, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
  elif network_name == 'inceptionv3':
    
    input_shape = (32,32,3)
    base_model = applications.inception_v3.InceptionV3(include_top = False, weights=weights, input_shape=input_shape, pooling='avg')
    preprocess_input = applications.inception_v3.preprocess_input
    
    x = base_model.output
    x = Dense(1024, activation='relu')(x) #Keras functional API
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model  = Model(inputs = base_model.inputs, outputs=predictions)
  elif network_name == 'mobilenetv2':
    
    input_shape = (32,32,3)
    base_model = applications.mobilenet_v2.MobileNetV2(input_shape=input_shape, alpha=1.0, include_top=False, weights=weights, pooling='avg')
    preprocess_input = applications.mobilenet_v2.preprocess_input
    
    x = base_model.output
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs = base_model.input, outputs=predictions)
  else:
    raise ValueError('{} is not a supported network'.format(network_name))
  return model, input_shape, preprocess_input

In [8]:
# select which model you would like to use here
# Currently you can use mobilenetv2, inceptionv3 or vgg16
model_to_use = 'mobilenetv2'
weights= 'imagenet'

model, image_shape, preprocess_input = network_library(model_to_use, 100, weights=weights)

model.summary()
print(image_shape)

C:\Users\gudim\Anaconda3\lib\site-packages\keras_applications\mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


9412608/9406464 [==============================] - 9s 1us/step
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 33, 33, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 16, 16, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 16, 16, 32)   128         Conv1[0][0]                      
_____________________________

In [9]:
model.compile(optimizer=SGD(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
history = model.fit(trainX, trainY, batch_size=16, epochs=20, verbose=True, validation_split=.1)

Train on 45000 samples, validate on 5000 samples
Epoch 1/20
44128/45000 [============================>.] - ETA: 19s - loss: 4.5272 - accuracy: 0.0426

KeyboardInterrupt: 